In [ ]:
%pylab inline
import scipy.interpolate as spi
import scipy.integrate as spint
import scipy.signal as spsig
import sys, os

sys.path.insert(0, os.path.abspath('../Rayleigh'))

import post_processing.rayleigh_diagnostics as rd
import post_processing.reference_tools as rt
import stelo.model_reader as mr

In [ ]:
rcParams['figure.dpi'] = 200

In [ ]:
def interp(r, v):
    prad = p.rmid[::-1] * mesa.rsol
    #You can also use 10**p.logR[::-1] or p.radius[::-1] instead of rmid[::-1], but rmid is the most accurate choice
    return np.interp(r, prad, v[::-1])

In [ ]:
p = mr.mesa_model('model.prof')

In [ ]:
R = p.r[-1]

In [ ]:
nr = 10000
r0 = 0.02 * R # in cm
r1 = 0.9 * R # in cm
rconv = p.r[np.nonzero(p.brunt() < 0)[0][0]] # in cm
radius = np.linspace(r0, r1, nr)

In [ ]:
print(f"{r0:.5e}, {rconv: .5e}, {r1:.5e}")

In [ ]:
r_MESA = p.r
density = p.intp.rho(radius)
temperature = p.intp.T(radius)
grav = p.intp.grav(radius)
cp = p.intp.cp(radius)
buoy = density * grav / cp
nu = np.ones_like(radius)
kappa = density**-0.5
eta = np.ones_like(radius)
hprofile = np.zeros_like(radius)
#N2 = p.intp.brunt(radius)
# use the value from MESA instead of the computed one
p.N2 = p.orig_data('brunt_N2')
N2 = p.intp.N2(radius)
dsdr = cp * N2 / grav

In [ ]:
plot(radius, kappa*1e12)
yscale('log')

In [ ]:
nu = (8e11 + (np.tanh((radius - 0.9 * radius[-1]) / (0.05 * radius[-1])) + 1) * 0.5e12) / 1e12
kappa = eta = nu

In [ ]:
hprofile = np.gradient(p.orig_data('conv_L_div_L') * p.luminosity, p.r) / (4 * np.pi * p.r**2)
hprofile = mr.interpolfuncs['linear'](p.r, hprofile)(radius)
hprofile_smooth = mr.h.smooth_data(hprofile, window_len=200)

In [ ]:
plt.plot(radius, hprofile)
plt.plot(radius, hprofile_smooth)
xlim(0., 2e10)

In [ ]:
my_ref = rt.equation_coefficients(radius)

my_ref.density = density
my_ref.buoy = buoy
my_ref.buoy_fact = 1.0

# There are all normalized to 1.
# They can be adjusted with the corresponding factors.
my_ref.nu = nu
my_ref.kappa = kappa
my_ref.eta = eta

my_ref.visc_fact = 1e12
my_ref.diff_fact = 1e12
my_ref.resist_fact = 1e12

my_ref.temperature = temperature
my_ref.p_fact = 1.0
my_ref.ds_dr = dsdr
my_ref.lorentz_fact = 0.25/np.pi

my_ref.heating = hprofile
# This can be overridden to boost the luminosity.
my_ref.luminosity = 1.0

In [ ]:
file_write='cref_from_MESA.dat'
my_ref.write(file_write)

In [ ]:
ma = p.r < rconv
vrms = np.sum(p.vrms[ma] * p.r[ma]**2) / np.sum(p.r[ma]**2)
plot(p.r[ma], p.vrms[ma])
axhline(vrms)

In [ ]:
vrms